# **Automotive DrivAer**

The DrivAer model is a standardized automotive geometry developed specifically for aerodynamic research. It represents a simplified yet realistic vehicle shape that captures essential aerodynamic features of modern passenger cars. The model comes in different configurations and the one used in this notebook is **F_D_wM_wW**. It features:

- Notchback top
- Detailed underbody
- Contains mirrors and wheels

This standardized geometry allows for consistent comparison of CFD results across different solvers and methodologies. The DrivAer model is particularly valuable for:

- Benchmarking solver performance
- Studying fundamental aerodynamic phenomena
- Developing and testing turbulence models

In this analysis, we investigate the aerodynamic characteristics of the DrivAer model under three yaw angles. We will post process obtained results to create a downloadable PDF report showcasing results such as typical aerodynamic forces, as well as various visualizations of the car.

The model prepared for this simulation is characterized by:
- 5.7 m node count mesh
- 40 m/s wind speed
- No wheel rotation
- Steady state

## Setting up the simulations

In order to run the simulations, we will walk our way through uploading the project and then defining the necessary parameters alongside some additional information that will be used for post processing.

We will start by importing Flow360 as well as its various subcomponents.

In [1]:
import flow360 as fl
from flow360.examples import DrivAer
from flow360.log import log
from flow360.log import log, set_logging_level

DrivAer.get_files()

log.log_to_file = False
set_logging_level("INFO")

Next we can upload the project from volume mesh.

In [2]:
project = fl.Project.from_volume_mesh(
    DrivAer.mesh_filename,
    name="Automotive DrivAer",
)
vm = project.volume_mesh

Output()

[14:04:27] INFO: VolumeMesh successfully submitted:                                                                
                   type   = Volume Mesh                                                                            
                   name   = Automotive DrivAer                                                                     
                   id     = vm-84de15d3-9af8-42a1-8569-de4b6b3df96d                                                
                   status = uploaded                                                                               
           

Output()

For the sake of convenience we can print out all of the boundaries in our mesh. This will make assigning those boundaries easier.

In [3]:
log.info("Volume mesh contains the following boundaries:")
for boundary in vm.boundary_names:
    log.info("Boundary: " + boundary)

[14:05:03] INFO: Volume mesh contains the following boundaries:

           INFO: Boundary: blk-1/wheel_tire

           INFO: Boundary: blk-1/BODY

           INFO: Boundary: blk-1/wheel_rim

           INFO: Boundary: blk-1/WT_ground_front

...

           INFO: Boundary: blk-1/WT_outlet

           INFO: Boundary: blk-1/WT_side1

           INFO: Boundary: blk-1/WT_side2

           INFO: Boundary: blk-1/WT_ground_close

           INFO: Boundary: blk-1/WT_ground_patch

>Note: *You can also take a look at your project in the webUI to see what boundaries are available. Additionally, you will be able to see how they are represented in the mesh.*

Here, we will group the previously printed out boundaries.

In [4]:
freestream_surfaces = ["blk-1/WT_side1", "blk-1/WT_side2", "blk-1/WT_inlet", "blk-1/WT_outlet"]
slip_wall_surfaces = ["blk-1/WT_ceiling", "blk-1/WT_ground_front", "blk-1/WT_ground"]
wall_surfaces = list(set(vm.boundary_names) - set(freestream_surfaces) - set(slip_wall_surfaces))

Our simulation will be defined with the following parameters.

In [5]:
with fl.SI_unit_system:
    params = fl.SimulationParams(
        meshing=None,
        reference_geometry=fl.ReferenceGeometry(area=2.17, moment_length=2.7862),
        operating_condition=fl.AerospaceCondition(velocity_magnitude=40),
        models=[
            fl.Wall(surfaces=[vm[i] for i in wall_surfaces], use_wall_function=True),
            fl.Freestream(
                surfaces=[vm[i] for i in freestream_surfaces],
            ),
            fl.SlipWall(
                surfaces=[vm[i] for i in slip_wall_surfaces],
            ),
        ],
        user_defined_fields=[
            fl.UserDefinedField(
                name="Cpx",
                expression="double prel = primitiveVars[4] - pressureFreestream;"
                + "double PressureForce_X = prel * nodeNormals[0]; "
                + "Cpx = PressureForce_X / (0.5 * MachRef * MachRef) / magnitude(nodeNormals);",
            ),
        ],
    )

           INFO: using: SI unit system for unit inference.

Since we want to do quite a bit of post processing for this simulation, we will create a more expansive outputs section and add it to the rest of the params.

In [6]:
with fl.SI_unit_system:
    outputs = [
        fl.SurfaceOutput(
            surfaces=vm["*"],
            output_fields=[
                "Cp",
                "Cf",
                "yPlus",
                "CfVec",
                "primitiveVars",
                "wall_shear_stress_magnitude",
                "Cpx",
            ],
        ),
        fl.SliceOutput(
            entities=[
                *[
                    fl.Slice(
                        name=f"slice_y_{name}",
                        normal=(0, 1, 0),
                        origin=(0, y, 0),
                    )
                    for name, y in zip(["0", "0_2", "0_4", "0_6", "0_8"], [0, 0.2, 0.4, 0.6, 0.8])
                ],
                *[
                    fl.Slice(
                        name=f"slice_z_{name}",
                        normal=(0, 0, 1),
                        origin=(0, 0, z),
                    )
                    for name, z in zip(
                        ["neg0_2", "0", "0_2", "0_4", "0_6", "0_8"],
                        [-0.2, 0, 0.2, 0.4, 0.6, 0.8],
                    )
                ],
            ],
            output_fields=["velocity", "velocity_x", "velocity_y", "velocity_z"],
        ),
        fl.IsosurfaceOutput(
            output_fields=["Cp", "Mach"],
            isosurfaces=[
                fl.Isosurface(
                    name="isosurface-cpt",
                    iso_value=-1,
                    field="Cpt",
                ),
            ],
        ),
        fl.ProbeOutput(
            entities=[fl.Point(name="point1", location=(10, 0, 1))],
            output_fields=["velocity"],
        ),
    ]

params.outputs = outputs

           INFO: using: SI unit system for unit inference.

### Running the simulations

With all simulation parameters prepared, we can initiate our simulations for three different beta angles (0, 5 and 10 degrees). We will append each case's data to a list, which will be necessary for report generation.

In [7]:
cases = []

for beta in [0, 5, 10]:
    params.operating_condition.beta = beta * fl.u.deg
    case_new = project.run_case(params=params, name=f"DrivAer 5.7M - beta={beta}")
    cases.append(case_new)

           INFO: using: SI unit system for unit inference.

[14:05:05] INFO: Successfully submitted:                                                                           
                   type   = Case                                                                                   
                   name   = DrivAer 5.7M - beta=0                                                                  
                   id     = case-9305d1d2-1fce-4d88-9654-a956aae1e276                                              
                   status = pending                                                                                
           

           INFO: using: SI unit system for unit inference.

[14:05:07] INFO: Successfully submitted:                                                                           
                   type   = Case                                                                                   
                   name   = DrivAer 5.7M - beta=5                                                                  
                   id     = case-70d6ce5b-9b8b-486e-99c2-7aaf3b4abe2b                                              
                   status = pending                                                                                
           

[14:05:08] INFO: using: SI unit system for unit inference.

[14:05:10] INFO: Successfully submitted:                                                                           
                   type   = Case                                                                                   
                   name   = DrivAer 5.7M - beta=10                                                                 
                   id     = case-7dd45edc-3f33-41a9-8317-da68acfe5a0e                                              
                   status = pending                                                                                
           

## Postprocessing

Once we have run the cases, setting up the post processing in the form of a report is going to be the next step.

Since the cases may still be running, we will make sure that they will finish before the report generation step by using `wait()`.

In [8]:
# wait until all cases finish running
for case in cases:
    case.wait()

Output()

Output()

Output()

### Flow360 Report

Report is a customizable document created using LaTeX that allows for consistent storage and presentation of obtained results.
It contains three main ways of presenting data:
- tabular data, e.g. case name, CD, CDA, CL
- 2D chart, e.g. plot of total CD vs pseudo step
- 3D chart, e.g. visualization of y+ distribution on the surfaces of the car
Other than this base functionality, the report can be customized using additional options such as captions for charts and formatting for tables.

Because our simulation domain contains many boundaries that irrelevant from the perspective of postprocessing results around the car, we will exclude them from visulizations. This will ensure that when `exclude` is specified, only `blk-1/BODY`, `blk-1/wheel_rim` and `blk-1/wheel_tire` boundaries are going to be taken into account.

In [9]:
exclude = ["blk-1/WT_ground_close", "blk-1/WT_ground_patch"]
size = "5.7M"

exclude += freestream_surfaces + slip_wall_surfaces

Some additional imports will be needed to create the report.

>Note: Importing units (u) here is only for the sake of convenience.

In [ ]:
from flow360 import u
from flow360.plugins.report.report import ReportTemplate
from flow360.plugins.report.report_items import (
    BottomCamera,
    Chart2D,
    Chart3D,
    FrontCamera,
    FrontLeftBottomCamera,
    FrontLeftTopCamera,
    Inputs,
    LeftCamera,
    RearCamera,
    RearLeftTopCamera,
    RearRightBottomCamera,
    Settings,
    Summary,
    Table,
    TopCamera,
)
from flow360.plugins.report.utils import (
    Average,
    DataItem,
    Delta,
    Expression,
    Variable,
    GetAttribute,
)

Let's position cameras based on predefined settings. These cameras will be used to take screenshots of our model.

In [11]:
top_camera = TopCamera(pan_target=(1.5, 0, 0), dimension=5, dimension_dir="width")
top_camera_slice = TopCamera(pan_target=(2.5, 0, 0), dimension=8, dimension_dir="width")
side_camera = LeftCamera(pan_target=(1.5, 0, 0), dimension=5, dimension_dir="width")
side_camera_slice = LeftCamera(pan_target=(2.5, 0, 1.5), dimension=8, dimension_dir="width")
rear_camera = RearCamera(dimension=2.5, dimension_dir="width")
front_camera = FrontCamera(dimension=2.5, dimension_dir="width")
bottom_camera = BottomCamera(pan_target=(1.5, 0, 0), dimension=5, dimension_dir="width")
front_left_bottom_camera = FrontLeftBottomCamera(
    pan_target=(1.5, 0, 0), dimension=5, dimension_dir="width"
)
rear_right_bottom_camera = RearRightBottomCamera(
    pan_target=(1.5, 0, 0), dimension=6, dimension_dir="width"
)
front_left_top_camera = FrontLeftTopCamera(
    pan_target=(1.5, 0, 0), dimension=6, dimension_dir="width"
)
rear_left_top_camera = RearLeftTopCamera(pan_target=(1.5, 0, 0), dimension=6, dimension_dir="width")

We can group those camera position to so that later, we simply use a predefined set instead of typing out each position every time.

In [12]:
cameras_geo = [
    top_camera,
    side_camera,
    rear_camera,
    bottom_camera,
    front_left_bottom_camera,
    rear_right_bottom_camera,
]
cameras_cp = [
    front_camera,
    front_left_top_camera,
    side_camera,
    rear_left_top_camera,
    rear_camera,
    bottom_camera,
    front_left_bottom_camera,
    rear_right_bottom_camera,
]

In order to better highlight the coefficient of pressure and its distribution on the surface of the car, we will assign specific limits for each camera position.

In [13]:
limits_cp = [(-1, 1), (-1, 1), (-1, 1), (-0.3, 0), (-0.3, 0), (-1, 1), (-1, 1), (-1, 1)]

#### Tabular data

We want to create a table containing some of the obtained results in our report. To do that we will create instances of `DataItem`, which can take in additional parameters other than the data itself. Field `operations` for example, allows to perform various operations on the data such as averaging, before including it in the report.

>Note: These operations are simply mathematical expressions as can be visible below.

In [14]:
avg = Average(fraction=0.1)
CD = DataItem(data="surface_forces/totalCD", exclude=exclude, title="CD", operations=avg)
CL = DataItem(data="surface_forces/totalCL", exclude=exclude, title="CL", operations=avg)
CDA = DataItem(
    data="surface_forces",
    exclude=exclude,
    title="CD*area",
    variables=[Variable(name="area", data="params.reference_geometry.area")],
    operations=[Expression(expr="totalCD * area"), avg],
)
CLf = DataItem(
    data="surface_forces",
    exclude=exclude,
    title="CLf",
    operations=[Expression(expr="1/2*totalCL + totalCMy"), avg],
)
CLr = DataItem(
    data="surface_forces",
    exclude=exclude,
    title="CLr",
    operations=[Expression(expr="1/2*totalCL - totalCMy"), avg],
)
CFy = DataItem(data="surface_forces/totalCFy", exclude=exclude, title="CS", operations=avg)
OWL = DataItem(
    data="volume_mesh/bounding_box",
    title="OWL",
    operations=[GetAttribute(attr_name="length")],
    exclude=exclude,
)
OWW = DataItem(
    data="volume_mesh/bounding_box",
    title="OWW",
    operations=[GetAttribute(attr_name="width")],
    exclude=exclude,
)
OWH = DataItem(
    data="volume_mesh/bounding_box",
    title="OWH",
    operations=[GetAttribute(attr_name="height")],
    exclude=exclude,
)

All of the previously created `DataItem`s can now be assembled into a list of statistical data alongside some additional entries that don't require special treatment, and can therefore be included by their path.

>Note: The `Table`s are split into two to avoid overflowing through the page.

In [15]:
statistical_data_general = [
    "volume_mesh/stats/n_nodes",
    "params/time_stepping/max_steps",
    OWL,
    OWW,
    OWH,
]
statistical_table_general = Table(
    data=statistical_data_general,
    section_title="General characteristics",
    formatter=[
        (
            None
            if d
            in [
                "params/reference_geometry/area",
                "volume_mesh/stats/n_nodes",
                "params/time_stepping/max_steps",
            ]
            else ".4f"
        )
        for d in statistical_data_general
    ],
)
statistical_data_forces = [
    "params/reference_geometry/area",
    CD,
    CDA,
    Delta(data=CD),
    CL,
    CLf,
    CLr,
    CFy,
]
statistical_table_forces = Table(
    data=statistical_data_forces,
    section_title="Statistical data",
    formatter=[
        (
            None
            if d
            in [
                "params/reference_geometry/area",
                "volume_mesh/stats/n_nodes",
                "params/time_stepping/max_steps",
            ]
            else ".4f"
        )
        for d in statistical_data_forces
    ],
)

#### Visualizations

Here we will define separate lists of `Chart3D`, which are essentially 3D visualizations (you can think of them as screenshots of your model). These come with a variety of options that can be used to customize the way data is shown.

One of the common things is that pretty much all of those charts contain either an exclude or include field. They provide control over what parts of the simulation domain will be excluded/included in the image.

>Note: If you took a peek at the charts you may have noticed that list comprehension can used to iterate over camera positions.

In [16]:
forces = [
    Chart2D(
        x="x_slicing_force_distribution/X",
        y="x_slicing_force_distribution/totalCumulative_CD_Curve",
        fig_name="totalCumulative_CD_Curve",
        background="geometry",
        exclude=exclude,
    ),
    Chart2D(
        x="surface_forces/pseudo_step",
        y="surface_forces/totalCD",
        section_title="Drag Coefficient",
        fig_name="cd_fig",
        exclude=exclude,
        focus_x=(1 / 3, 1),
    ),
]
geometry_screenshots = [
    Chart3D(
        section_title="Geometry",
        items_in_row=2,
        force_new_page=True,
        show="boundaries",
        camera=camera,
        exclude=exclude,
        fig_name=f"geo_{i}",
    )
    for i, camera in enumerate(cameras_geo)
]
cpt_screenshots = [
    Chart3D(
        section_title="Isosurface, Cpt=-1",
        items_in_row=2,
        force_new_page=True,
        show="isosurface",
        iso_field="Cpt",
        exclude=exclude,
        camera=camera,
    )
    for camera in cameras_cp
]
cfvec_screenshots = [
    Chart3D(
        section_title="CfVec",
        items_in_row=2,
        force_new_page=True,
        show="boundaries",
        field="CfVec",
        mode="lic",
        limits=(1e-4, 10),
        is_log_scale=True,
        exclude=exclude,
        camera=camera,
    )
    for camera in cameras_cp
]
y_slices_screenshots = [
    Chart3D(
        section_title=f"Slice velocity y={y}",
        items_in_row=2,
        force_new_page=True,
        show="slices",
        include=[f"slice_y_{name}"],
        field="velocity",
        limits=(0 * u.m / u.s, 50 * u.m / u.s),
        camera=side_camera_slice,
        fig_name=f"slice_y_{name}",
    )
    for name, y in zip(["0", "0_2", "0_4", "0_6", "0_8"], [0, 0.2, 0.4, 0.6, 0.8])
]
y_slices_lic_screenshots = [
    Chart3D(
        section_title=f"Slice velocity LIC y={y}",
        items_in_row=2,
        force_new_page=True,
        show="slices",
        include=[f"slice_y_{name}"],
        field="velocityVec",
        mode="lic",
        limits=(0 * u.m / u.s, 50 * u.m / u.s),
        camera=side_camera_slice,
        fig_name=f"slice_y_vec_{name}",
    )
    for name, y in zip(["0", "0_2", "0_4", "0_6", "0_8"], [0, 0.2, 0.4, 0.6, 0.8])
]
z_slices_screenshots = [
    Chart3D(
        section_title=f"Slice velocity z={z}",
        items_in_row=2,
        force_new_page=True,
        show="slices",
        include=[f"slice_z_{name}"],
        field="velocity",
        limits=(0 * u.m / u.s, 50 * u.m / u.s),
        camera=top_camera_slice,
        fig_name=f"slice_z_{name}",
    )
    for name, z in zip(["neg0_2", "0", "0_2", "0_4", "0_6", "0_8"], [-0.2, 0, 0.2, 0.4, 0.6, 0.8])
]
y_plus_screenshots = [
    Chart3D(
        section_title="y+",
        items_in_row=2,
        show="boundaries",
        field="yPlus",
        exclude=exclude,
        limits=(0, 5),
        camera=camera,
        fig_name=f"yplus_{i}",
    )
    for i, camera in enumerate([top_camera, bottom_camera])
]
cp_screenshots = [
    Chart3D(
        section_title="Cp",
        items_in_row=2,
        show="boundaries",
        field="Cp",
        exclude=exclude,
        limits=limits,
        camera=camera,
        fig_name=f"cp_{i}",
    )
    for i, (limits, camera) in enumerate(zip(limits_cp, cameras_cp))
]
cpx_screenshots = [
    Chart3D(
        section_title="Cpx",
        items_in_row=2,
        show="boundaries",
        field="Cpx",
        exclude=exclude,
        limits=(-0.3, 0.3),
        camera=camera,
        fig_name=f"cpx_{i}",
    )
    for i, camera in enumerate(cameras_cp)
]
wall_shear_screenshots = [
    Chart3D(
        section_title="Wall shear stress magnitude",
        items_in_row=2,
        show="boundaries",
        field="wallShearMag",
        exclude=exclude,
        limits=(0 * u.Pa, 5 * u.Pa),
        camera=camera,
        fig_name=f"wallShearMag_{i}",
    )
    for i, camera in enumerate(cameras_cp)
]

#### Report assembly

Once we have all of the components of our report, we can create a `ReportTemplate` that will allow us to then generate the report itself. In our case, we will also add two `Chart2D`s related to drag coefficient.

In [17]:
report_template = ReportTemplate(
    title="Aerodynamic analysis of DrivAer",
    items=[
        Summary(),
        Inputs(),
        statistical_table_general,
        statistical_table_forces,
        *forces,
        *geometry_screenshots,
        *cp_screenshots,
        *cpx_screenshots,
        *cpt_screenshots,
        *y_slices_screenshots,
        *y_slices_lic_screenshots,
        *z_slices_screenshots,
        *y_plus_screenshots,
        *wall_shear_screenshots,
    ],
    settings=Settings(dpi=150),
)

All that is left now is to create the report by calling the `create_in_cloud()` function, then downloading it. To ensure that the download will only start after the report has actually been generated, we use `wait()`.

In [ ]:
report = report_template.create_in_cloud(
    f"{size}-{len(cases)}cases-slices-using-groups-Cpt, Cpx, wallShear, dpi=default",
    cases,
)

report.wait()
report.download("report.pdf")

Output()

Output()

[14:15:44] INFO: Saved to report.pdf

'report.pdf'